In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pyxona import File
import quantities as pq
import ripple_detection as rd
from scipy.signal import butter, lfilter
import tools

In [2]:
fnames = [
#    ['KKL196', 201015, 1, 'KKL196_201015_1_hpc', 'KKL196_201015_1_ctx'],
    ['KKL196', 201015, 2, 'KKL196_201015_2_hpc', 'KKL196_201015_2_ctx'],
#    ['KKL196', 201016, 3, 'KKL196_201016_3_hpc', 'KKL196_201016_3_ctx'],
#    ['KKL196', 201017, 3, 'KKL196_201017_3_hpc', 'KKL196_201017_3_ctx'],
#    ['KKL196', 201018, 2, 'KKL196_201018_2_hpc', 'KKL196_201018_2_ctx'],
#    ['KKL196', 201020, 1, 'KKL196_201020_1_hpc', 'KKL196_201020_1_ctx'],  
]

df_fnames = pd.DataFrame(fnames, columns=['animal', 'date', 'session', 'fname_hpc', 'fname_ctx'])


path = "/home/jovyan/work/RippleSpindleCoupling/test_data_renamed/"

sampling_rate = 4800
sampling_interval = 1./sampling_rate

>For ripple detection (Boyce et al., 2016; Nakashiba et al., 2009), the LFP obtained from CA1
pyramidal cell layer was first band-pass filtered (100–250 Hz), and amplitude was calculated using
the Hilbert transform. Ripple windows were characterized as signals that exceed the amplitude
threshold (three times the standard deviation). Signals that were less than 50 ms apart were merged.

> For spindle detection (Eschenko et al., 2006; Phillips et al., 2012), the LFP obtained from ACC
was band-pass filtered (12–15 Hz), and amplitude was calculated using the Hilbert transform. Spindle
windows were characterized as signals that exceed the amplitude threshold (two times the standard
deviation), with minimum and maximum duration of 200 and 2000 ms, respectively. Signals that are
less than 100 ms apart were merged.

In [3]:
params_spindles = {
    'freq_min': 12,
    'freq_max': 15,
    'buttw_ord': 2,
    'dur_min': 0.15,
    'dur_max': 0.5,
    'dur_merge': 0.1,
    'ampl_thresh': 1.5
}
params_ripples = {
    'freq_min': 100,
    'freq_max': 250,
    'buttw_ord': 5,
    'dur_min': 0.,
    'dur_max': 1.,
    'dur_merge': 0.05,
    'ampl_thresh': 3.
}

In [4]:
for index, row in df_fnames.iterrows():
    lfp = []

    for name_i in ['fname_hpc', 'fname_ctx']:
        fname_i = row[name_i]+'.set'
        f = File(path+fname_i)
        lfp_i = f.analog_signals[0]
        assert lfp_i.sample_rate == sampling_rate
        lfp.append(lfp_i.signal)

    evnts_rppls, magn_rppls = tools.event_detection(
        lfp[0],
        sampling_rate,
        return_magn=True,
        **params_ripples)
    evnts_spndls, magn_spndls = tools.event_detection(
        lfp[1],
        sampling_rate,
        return_magn=True,
        **params_spindles)    


/opt/conda/lib/python3.8/site-packages/pyxona/core.py:589: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  data = np.fromfile(f, dtype=sample_dtype, count=sample_count)


In [5]:
evnts_spndls

[(159.11666666666667, 159.37395833333335),
 (373.870625, 374.075625),
 (375.27083333333337, 375.5470833333334),
 (382.65541666666667, 382.89625),
 (386.2229166666667, 386.43708333333336),
 (387.20125, 387.5116666666667),
 (544.7541666666667, 545.0914583333333),
 (568.4414583333333, 568.6489583333333),
 (576.6070833333333, 576.8062500000001),
 (618.7914583333334, 618.975),
 (628.6929166666667, 629.0804166666667),
 (637.2933333333334, 637.6464583333334),
 (647.8260416666667, 648.0960416666667),
 (649.218125, 649.390625),
 (737.0770833333333, 737.4920833333334),
 (737.9433333333334, 738.1577083333334),
 (742.1329166666667, 742.5977083333333),
 (780.3687500000001, 780.5904166666667),
 (781.8583333333333, 782.0558333333333),
 (782.7791666666667, 783.2297916666668),
 (867.1177083333334, 867.5450000000001),
 (930.8047916666667, 931.031875),
 (932.3743750000001, 932.5247916666667),
 (936.2635416666667, 936.4629166666667),
 (993.2825, 993.5595833333334),
 (993.6602083333333, 993.8279166666667),

In [6]:
evnts_rppls

[(0.005833333333333334, 0.009791666666666667),
 (0.06104166666666667, 0.06541666666666666),
 (0.13270833333333334, 0.13708333333333333),
 (0.3902083333333334, 0.393125),
 (0.9014583333333334, 0.9045833333333334),
 (2.0810416666666667, 2.081666666666667),
 (2.1375, 2.1425),
 (2.39625, 2.3975),
 (2.565, 2.566666666666667),
 (5.199583333333334, 5.20375),
 (6.0791666666666675, 6.080625),
 (7.710625, 7.711875000000001),
 (9.787291666666667, 9.790208333333334),
 (10.590416666666668, 10.595),
 (10.765, 10.780208333333334),
 (12.993958333333333, 12.995416666666667),
 (14.040833333333333, 14.043958333333334),
 (15.162916666666668, 15.168541666666668),
 (15.263333333333334, 15.263541666666667),
 (15.582916666666668, 15.5875),
 (16.229791666666667, 16.603333333333335),
 (16.7325, 16.853958333333335),
 (16.910833333333333, 16.914166666666667),
 (16.96979166666667, 17.057083333333335),
 (17.406458333333333, 17.41),
 (18.32625, 18.397916666666667),
 (18.51875, 18.572708333333335),
 (19.9893750000000